# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract image data 

## Read dataframe

In [5]:
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/',
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [7]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [8]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200914_220904_bsize64_spec_test_128_withspec


In [9]:
# main_dir=''

In [10]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

In [11]:
print(df.shape)
df.head()


(140640, 20)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
0,0,0,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,188.919295,4.752520,0.013036,193.684851,12.402487,26.132894,17735.385188,2.903151e+07,0,0,"[0.00021404027994051573, 0.05836540469577179, ...","[5.26696921090502e-05, 0.00262919138735563, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[56721116.45792523, 2266.702792803757, 1044.62...","[49429.14483162096, 68.58417266486025, 37.4961...",0-0
1,0,1,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,238.548875,5.352387,0.013186,243.914448,14.278033,30.975412,8854.748195,1.319088e+07,0,0,"[4.693865788169204e-05, 0.029790490911596794, ...","[1.1144842749475391e-05, 0.0016189433612405334...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52189522.432882756, 5458.261089618551, 1901.3...","[98812.95971382115, 176.00371938674039, 61.038...",0-1
2,0,2,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,305.103441,6.876216,0.013186,311.992843,16.582324,35.608395,5586.276721,9.477785e+06,0,0,"[7.510185261070727e-06, 0.013805061600015982, ...","[3.635852805382416e-06, 0.000819668697804994, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[47665268.64489394, 12155.791003553197, 3623.0...","[155416.24249774014, 296.77206865816515, 96.47...",0-2
3,0,3,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,400.746239,9.788279,0.013186,410.547704,19.579865,38.989692,4988.186476,1.218449e+07,0,0,"[0.0, 0.0048859119490545985, 0.133558422441453...","[0.0, 0.00034019621275051194, 0.00247884888666...","[-1.031746031584782, -0.6161616169043975, -0.3...","[43211358.01204732, 20322.56252185628, 5744.85...","[174205.08008122293, 490.7121617184416, 143.97...",0-3
4,0,4,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,533.948721,15.445696,0.013186,549.407603,23.508774,42.566878,5132.435991,1.466475e+07,0,0,"[0.0, 0.0008910298361198608, 0.068541437446182...","[0.0, 7.789009261683955e-05, 0.001622468667198...","[-1.031746031584782, -0.6161616169043975, -0.3...","[38971199.548148364, 28604.48045221006, 7959.0...","[165333.33079042763, 565.3834694462739, 158.65...",0-4


### View best epochs

#### Locations with best chi_sqr

In [12]:
chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]

,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
1245,0,1245,9.048651,11113.858737,0.013036,11122.920424,144.754960,40.421721,1516.756260,2.788719e+09
0,0,0,188.919295,4.752520,0.013036,193.684851,12.402487,26.132894,17735.385188,2.903151e+07
1187,0,1187,14.869902,9563.043688,0.008758,9577.922348,128.857540,38.165398,1196.557081,1.614321e+09
0,0,0,188.919295,4.752520,0.013036,193.684851,12.402487,26.132894,17735.385188,2.903151e+07
0,0,0,188.919295,4.752520,0.013036,193.684851,12.402487,26.132894,17735.385188,2.903151e+07
134250,38,134250,16.832127,14265.638640,0.013186,14282.483953,204.246479,4.528781,1219.723411,5.641273e+11
54,0,54,158.678194,5211.375738,31.727857,5401.781789,99.740831,82.321793,807.122257,5.077054e+07
2,0,2,305.103441,6.876216,0.013186,311.992843,16.582324,35.608395,5586.276721,9.477785e+06


In [20]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

In [21]:
df_sliced=f_get_best_chisqr_models(df,cutoff=0.2)
print(df_sliced.shape)

(19518, 20)


In [22]:
df_sliced.sort_values(by=['chi_1','chi_spec1'])[['epoch','step','chi_1','chi_spec1']].head(5)

,epoch,step,chi_1,chi_spec1
35,0,35,4373.589823,1219.675854
36,0,36,4438.073570,1147.549407
37,0,37,4451.607392,1112.350644
38,0,38,4451.665969,1020.928243
39,0,39,4535.165815,1013.695162


In [23]:
### Plot chi-sqr values
df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [24]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

In [25]:
df[df.num_vlarge>10].tail()

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label


## Compare samples

In [26]:
# df_sliced.sort_values(by=['chi_spec1','chi_1']).head(5)
# df_sliced.sort_values(by=['chi_1','chi_spec1']).head(5)

In [27]:
best_idx=[]
best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec1']).head(2).index)
best_idx.append(df_sliced.sort_values(by=['chi_spec1','chi_1']).head(2).index)

best_idx=[i for j in best_idx for i in j]
print(best_idx)

df_best=df_sliced.loc[best_idx]
print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])
df_best

[35, 36, 54, 52]
[(0, 35), (0, 36), (0, 54), (0, 52)]


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
35,0,35,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,538.973332,3834.603305,0.013186,4373.589823,89.777276,86.743845,1219.675854,2.603502e+07,0,0,"[0.0, 0.0012981891652077441, 0.007274776703390...","[0.0, 8.224240712745884e-05, 9.357480583108897...","[-1.031746031584782, -0.6161616169043975, -0.3...","[9657467.00357914, 37473.37834215164, 15267.07...","[56579.64159609792, 946.379936112115, 346.0094...",0-35
36,0,36,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,509.009950,3929.050434,0.013186,4438.073570,90.656120,76.798263,1147.549407,3.198437e+07,0,0,"[0.0, 0.002383947376108766, 0.0058385729839749...","[0.0, 9.59230098112155e-05, 0.0001144683710851...","[-1.031746031584782, -0.6161616169043975, -0.3...","[9504702.692451477, 39994.643642902374, 16550....","[48131.91923355958, 902.1397473202502, 310.698...",0-36
54,0,54,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,158.678194,5211.375738,31.727857,5401.781789,99.740831,82.321793,807.122257,5.077054e+07,2,0,"[0.0007810592671513557, 0.014138460181080705, ...","[5.263830934268829e-05, 0.00017650871484092178...","[-1.031746031584782, -0.6161616169043975, -0.3...","[7662084.639343262, 53939.17410469055, 26008.8...","[28922.232491906718, 915.0093844252568, 369.05...",0-54
52,0,52,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,168.738177,5138.987370,13.124763,5320.850311,99.093487,82.531194,816.645678,4.348380e+07,0,0,"[0.00037926435568407175, 0.01191973688054383, ...","[4.0723174536509075e-05, 0.0001379779522348443...","[-1.031746031584782, -0.6161616169043975, -0.3...","[7754519.759607315, 52418.12523841858, 24940.3...","[31463.91323786883, 1057.9386897543595, 406.61...",0-52


## Plot pixel intensity and spectrum

In [28]:
def f_plot_hist_spec_combined(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label='{0}_{1}_{2}'.format(i,row.epoch,row.step)
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    
    


In [29]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:4000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


(4000, 128, 128)


In [30]:
f_plot_hist_spec_combined(df_best,dict_val)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## View image block

In [32]:
# df[df.epoch>10]

In [16]:
def f_get_img(df,epoch,step):
    df_temp=df[(df.epoch==epoch)&(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,38,134250)

In [17]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df_sliced[df_sliced.epoch==4]['step'].values

In [ ]:
df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]